In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re

import nltk
from nltk import tokenize
from collections import Counter

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn import metrics

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVC
from xgboost import XGBClassifier

In [2]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [3]:
df = pd.read_csv('volk_polo_used_dataset.csv', index_col=0)

In [4]:
df.head()

,city,model,photos,vin,владельцы,год выпуска,двигатель,коробка,кузов,налог,...,госномер,онлайн-показ,1 владелец,дтп ненайдены,почти как новый,гарантия,на гарантии,продаёт собственник,медленно теряет вцене,кузов №
0,архангельск,volkswagen polo v,1.000,xw8**************,2владельца,2012,1.6 л / 105л.с. / бензин,механическая,седан,2625₽ / год,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,санкт-петербург,volkswagen polo v рестайлинг,1.000,xw8**************,2владельца,2017,1.6 л / 90л.с. / бензин,механическая,седан,2160₽ / год,...,******|198,1.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,минеральные воды,volkswagen polo v,1.000,xw8**************,1владелец,2014,1.6 л / 105л.с. / бензин,механическая,седан,1575₽ / год,...,NaN,NaN,1.000,1.000,NaN,NaN,NaN,NaN,NaN,NaN
3,северская,volkswagen polo v рестайлинг,1.000,xw8**************,1владелец,2018,1.6 л / 110л.с. / бензин,механическая,седан,2750₽ / год,...,******|123,NaN,1.000,NaN,1.000,NaN,NaN,NaN,NaN,NaN
4,бирск,volkswagen polo v рестайлинг,1.000,xw8**************,2владельца,2019,1.6 л / 110л.с. / бензин,автоматическая,седан,3850₽ / год,...,******|702,NaN,NaN,1.000,NaN,до марта 2030,1.000,NaN,NaN,NaN


In [5]:
about = df.loc[~((df['описание'].isna()) | (df['описание'] == '')), 'описание']

In [6]:
about

0       ******** аксель-норд – официальный дилер toyot...
1       автомобиль купил у од, предыдущий владелец жен...
2       приобретался у официального дилера. 1 владелец...
3       фoльксвагeн полo, отличный пpостой сeмейный aв...
4       продам фольксваген поло в идиальном состоянии,...
                              ...                        
2909    преимущества этого автомобиля: ∙\tодин владеле...
2910    продаю  volkswagen polo,выпуска 2016 года!авто...
2911    сигнализация.кондиционер.камера заднего хода ....
2912    автомобиль в хорошем состоянии. комплектация: ...
2913    продаю верного друга. по птс я третий хозяин, ...
Name: описание, Length: 2841, dtype: object

In [7]:
price = df['цена'].apply(lambda x: str(x).replace('₽', '').strip()).astype('int')

In [8]:
price.describe()

count      2914.000
mean     605736.531
std      198975.533
min       30000.000
25%      456699.250
50%      610000.000
75%      750000.000
max     1080000.000
Name: цена, dtype: float64

In [9]:
price = price.iloc[about.index]

### About Preprocessing

In [10]:
from string import punctuation
from razdel import tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from natasha import MorphVocab, Doc, Segmenter, NewsEmbedding, NewsMorphTagger

### Приведем все к нижнему регистру

In [11]:
about = about.str.lower()

### Уберем лишние символы

In [12]:
pat = r'[^a-zA-zа-яА-Я\-]'
about_no_special_chars = about.apply(lambda x: re.sub(pat, ' ', x).replace('  ', ' ').replace(' - ', ' ').strip())

### Токенизируем

In [13]:
tokenized_about = about_no_special_chars.apply(lambda x: [y.text for y in tokenize(x)])

In [14]:
tokenized_about

0       [аксель-норд, официальный, дилер, toyota, в, а...
1       [автомобиль, купил, у, од, предыдущий, владеле...
2       [приобретался, у, официального, дилера, владел...
3       [ф, o, льксваг, e, н, пол, o, отличный, п, p, ...
4       [продам, фольксваген, поло, в, идиальном, сост...
                              ...                        
2909    [преимущества, этого, автомобиля, один, владел...
2910    [продаю, volkswagen, polo, выпуска, года, авто...
2911    [сигнализация, кондиционер, камера, заднего, х...
2912    [автомобиль, в, хорошем, состоянии, комплектац...
2913    [продаю, верного, друга, по, птс, я, третий, х...
Name: описание, Length: 2841, dtype: object

### Уберем стоп-слова и одиночные буквы

In [15]:
stop_words = stopwords.words('russian') + stopwords.words('english')

In [16]:
clean_about = tokenized_about.apply(lambda x: [y for y in x if y not in stop_words and len(y) > 1])

In [17]:
clean_about

0       [аксель-норд, официальный, дилер, toyota, арха...
1       [автомобиль, купил, од, предыдущий, владелец, ...
2       [приобретался, официального, дилера, владелец,...
3       [льксваг, пол, отличный, остой, мейный, втомоб...
4       [продам, фольксваген, поло, идиальном, состоян...
                              ...                        
2909    [преимущества, автомобиля, владелец, сервисная...
2910    [продаю, volkswagen, polo, выпуска, года, авто...
2911    [сигнализация, кондиционер, камера, заднего, х...
2912    [автомобиль, хорошем, состоянии, комплектация,...
2913    [продаю, верного, друга, птс, третий, хозяин, ...
Name: описание, Length: 2841, dtype: object

### Лемматизация

In [18]:
morph_vocab = MorphVocab()
segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
    
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text_list):
    doc = Doc(' '.join(text_list))
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)

    # лемматизация русских слов
    for token in doc.tokens:    
        token.lemmatize(morph_vocab)
    
    result = [x.lemma for x in doc.tokens]
    
    # лемматизация английских слов
    result = [lemmatizer.lemmatize(x) for x in result]

    return result

In [19]:
lemmatized_about = clean_about.apply(lambda x: lemmatize_text(x))

In [20]:
lemmatized_about

0       [аксель-норд, официальный, дилер, toyota, арха...
1       [автомобиль, купить, од, предыдущий, владелец,...
2       [приобретаться, официальный, дилер, владелец, ...
3       [льксваг, пол, отличный, остой, мейный, втомоб...
4       [продать, фольксваген, поло, идиальный, состоя...
                              ...                        
2909    [преимущество, автомобиль, владелец, сервисный...
2910    [продавать, volkswagen, polo, выпуск, год, авт...
2911    [сигнализация, кондиционер, камера, задний, хо...
2912    [автомобиль, хороший, состояние, комплектация,...
2913    [продавать, верный, друг, птс, третий, хозяин,...
Name: описание, Length: 2841, dtype: object

In [21]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [22]:
data = lemmatized_about.to_list()

In [23]:
# Create Dictionary
id2word = corpora.Dictionary(data)
# Create Corpus
texts = data.copy()
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 4), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 2), (12, 1), (13, 1), (14, 1), (15, 2), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 2), (22, 4), (23, 2), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 2), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 2), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 2), (55, 1), (56, 1), (57, 1), (58, 3), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1)]]


In [24]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('toyota', 1),
  ('trade-in', 1),
  ('авто', 1),
  ('автокресло', 1),
  ('автомобиль', 4),
  ('аксель-норд', 1),
  ('активный', 1),
  ('антиблокировочный', 1),
  ('антипробуксовочный', 1),
  ('архангельск', 1),
  ('аудиоподготовка', 1),
  ('безопасность', 2),
  ('блокировка', 1),
  ('бортовой', 1),
  ('водитель', 1),
  ('выкуп', 2),
  ('дальнейший', 1),
  ('дверь', 1),
  ('детский', 1),
  ('дилер', 1),
  ('диск', 1),
  ('дополнительный', 2),
  ('задний', 4),
  ('замок', 2),
  ('иммобилайзер', 1),
  ('интересовать', 1),
  ('информация', 1),
  ('комплектация', 1),
  ('компьютер', 1),
  ('кредит', 1),
  ('кредитный', 1),
  ('крепление', 1),
  ('любой', 1),
  ('мочь', 1),
  ('наличие', 1),
  ('наш', 2),
  ('обивка', 1),
  ('обмен', 1),
  ('оборудование', 1),
  ('обслуживание', 1),
  ('отдел', 1),
  ('официальный', 1),
  ('пассажир', 1),
  ('пепельница', 1),
  ('передний', 1),
  ('подголовник', 1),
  ('подушка', 2),
  ('покупка', 1),
  ('приезжать', 1),
  ('прикуриватель', 1),
  ('пробег'

In [25]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=3, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [26]:
lda_model.print_topics()

[(0,
  '0.019*"состояние" + 0.014*"машина" + 0.012*"автомобиль" + 0.011*"резина" + 0.011*"новый" + 0.010*"авто" + 0.010*"торг" + 0.010*"комплект" + 0.009*"хороший" + 0.009*"отличный"'),
 (1,
  '0.024*"автомобиль" + 0.020*"задний" + 0.019*"сидение" + 0.016*"система" + 0.015*"руль" + 0.015*"обогрев" + 0.013*"зеркало" + 0.013*"передний" + 0.012*"безопасность" + 0.012*"кредит"'),
 (2,
  '0.075*"автомобиль" + 0.021*"ваш" + 0.020*"пробег" + 0.019*"кредит" + 0.010*"наш" + 0.010*"продажа" + 0.010*"обмен" + 0.009*"юридический" + 0.008*"покупка" + 0.008*"условие"')]

In [27]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.889314747843517

Coherence Score:  0.6331719783396028


In [28]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=           x      y  topics  cluster   Freq
topic                                      
0     -0.231 -0.037       1        1 37.296
2      0.154 -0.148       2        1 33.920
1      0.077  0.185       3        1 28.784, topic_info=             Term     Freq    Total Category  logprob  loglift
4      автомобиль 6234.000 6234.000  Default   30.000   30.000
22         задний 1380.000 1380.000  Default   29.000   29.000
57        сидение 1290.000 1290.000  Default   28.000   28.000
58        система  823.000  823.000  Default   27.000   27.000
29         кредит 1680.000 1680.000  Default   26.000   26.000
..            ...      ...      ...      ...      ...      ...
138   кондиционер  330.265  631.696   Topic3   -5.003    0.597
27   комплектация  394.409  963.021   Topic3   -4.826    0.353
216      подогрев  376.554  878.773   Topic3   -4.872    0.398
20           диск  293.794  734.765   Topic3   -5.120    0.329
152           птс  292.999  784.496   Topic3   -5.123    0.261

[216 rows x 6 columns], token_table=      Topic  Freq                    Term
term                                     
225       1 0.078                     aux
225       3 0.918                     aux
1437      3 0.998                      cd
292       2 0.998                     cme
2419      3 0.994                 favorit
...     ...   ...                     ...
73        3 0.993              штамповать
661       3 0.997           электропривод
74        3 0.998  электростеклоподъемник
328       2 0.829             юридический
328       3 0.170             юридический

[289 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2])

In [29]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(
            corpus=corpus,
            id2word=dictionary,
            num_topics=num_topics, 
            random_state=100,
            update_every=1,
            chunksize=100,
            passes=10,
            alpha='auto',
            per_word_topics=True)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [30]:
# %%time
# model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data, start=2, limit=10, step=1)

In [31]:
# limit=10; start=2; step=1;
# x = range(start, limit, step)
# plt.plot(x, coherence_values)
# plt.xlabel("Num Topics")
# plt.ylabel("Coherence score")
# plt.legend(("coherence_values"), loc='best')
# plt.show()

In [32]:
df = pd.DataFrame()

In [33]:
for i in range(0, len(corpus)):
    topics = lda_model.get_document_topics(corpus[i])
    
    list_of_topics = [0.0, 0.0, 0.0]
    
    for topic in topics:
        topic_name = topic[0]
        topic_value = topic[1]
        
        list_of_topics[topic_name] = topic_value
    
    df = df.append(pd.Series(list_of_topics), ignore_index=True)

In [34]:
df

,0,1,2
0,0.000,0.733,0.265
1,0.900,0.023,0.077
2,0.000,0.994,0.000
3,0.960,0.000,0.039
4,0.994,0.000,0.000
...,...,...,...
2836,0.000,0.507,0.491
2837,0.546,0.453,0.000
2838,0.029,0.958,0.013
2839,0.992,0.000,0.000


In [35]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn import metrics

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVC
from xgboost import XGBClassifier

In [36]:
price

0        325000
1        665000
2        660000
3        750000
4       1000000
         ...   
2909     690000
2910     576000
2911     470000
2912     520000
2913     440000
Name: цена, Length: 2841, dtype: int64

In [40]:
price.index

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            2904, 2905, 2906, 2907, 2908, 2909, 2910, 2911, 2912, 2913],
           dtype='int64', length=2841)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df, price, test_size=0.25)

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def dataframe_metrics(y_test,y_pred):
    stats = [
       metrics.mean_absolute_error(y_test, y_pred),
       np.sqrt(metrics.mean_squared_error(y_test, y_pred)),
       metrics.r2_score(y_test, y_pred),
       mean_absolute_percentage_error(y_test, y_pred)
    ]
    return stats

In [ ]:
measured_metrics = pd.DataFrame({"error_type":["MAE", "RMSE", "R2", "MAPE"]})
measured_metrics.set_index("error_type")

In [ ]:
median_train = y_train.median()
baseline = np.array([median_train] * len(y_test))
measured_metrics['baseline'] = dataframe_metrics(y_test, baseline)
measured_metrics

In [ ]:
rf_reg = RandomForestRegressor()
rf_reg.fit(X_train, y_train)

In [ ]:
measured_metrics["rf_reg"] = dataframe_metrics(y_test, rf_reg.predict(X_test))
measured_metrics